<a href="https://colab.research.google.com/github/abhishekshakya/seq-2-seq-for-neural-machine-translation-english-to-hindi-/blob/master/machineTranlation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
from google.colab import drive
drive.mount('/content/gdrive')

!cp '/content/gdrive/My Drive/parallelCorpus/data.csv' 'data.csv'

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive


In [1]:
import pandas as pd
import numpy as np
import re
from tqdm.notebook import tqdm

In [2]:
df = pd.read_csv('data.csv')
df.head()

,source,english_sentence,hindi_sentence
0,ted,politicians do not have permission to do what ...,"राजनीतिज्ञों के पास जो कार्य करना चाहिए, वह कर..."
1,ted,"I'd like to tell you about one such child,",मई आपको ऐसे ही एक बच्चे के बारे में बताना चाहू...
2,indic2012,This percentage is even greater than the perce...,यह प्रतिशत भारत में हिन्दुओं प्रतिशत से अधिक है।
3,ted,what we really mean is that they're bad at not...,हम ये नहीं कहना चाहते कि वो ध्यान नहीं दे पाते
4,indic2012,.The ending portion of these Vedas is called U...,इन्हीं वेदों का अंतिम भाग उपनिषद कहलाता है।


In [3]:
df.drop(columns=['source'],inplace=True)

In [4]:
df = df.iloc[:10000,:]
df.head()

,english_sentence,hindi_sentence
0,politicians do not have permission to do what ...,"राजनीतिज्ञों के पास जो कार्य करना चाहिए, वह कर..."
1,"I'd like to tell you about one such child,",मई आपको ऐसे ही एक बच्चे के बारे में बताना चाहू...
2,This percentage is even greater than the perce...,यह प्रतिशत भारत में हिन्दुओं प्रतिशत से अधिक है।
3,what we really mean is that they're bad at not...,हम ये नहीं कहना चाहते कि वो ध्यान नहीं दे पाते
4,.The ending portion of these Vedas is called U...,इन्हीं वेदों का अंतिम भाग उपनिषद कहलाता है।


In [5]:
def cleanerEng(x):
  x = str(x)
  x = x.lower()
  x = re.sub(r'[^a-z0-9]+',' ',x)
  return x

def cleanerHindi(x):
  x = str(x)
  x = re.sub(r'[-.।|,?;:<>&$₹]+',' ',x)
  return x

In [6]:
df.iloc[:,0] = df['english_sentence'].apply(func=cleanerEng)
df.iloc[:,1] = df['hindi_sentence'].apply(func= cleanerHindi)
df.iloc[:,0] = df['english_sentence'].apply(func= lambda x : (str(x).split()))
df.iloc[:,1] = df['hindi_sentence'].apply(func= lambda x : (str(x).split()))

In [7]:
def addTokens(x,start=False):
  x.append('<END>')
  if start:
    x.insert(0,'<START>')
  return list(x)

In [8]:
df.iloc[:,0] = df['english_sentence'].apply(func= addTokens,start=False)
df.iloc[:,1] = df['hindi_sentence'].apply(func= addTokens,start=True)

In [9]:
df.iloc[79,1]

['<START>',
 'जो',
 'वह',
 'सीखता',
 'है',
 'या',
 'पूर्व',
 'क्रमादेशित',
 'होता',
 'है',
 '<END>']

In [10]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import random

In [11]:
data = df.values

In [12]:
data[:,1].shape

(10000,)

In [13]:
class vocab:

  def __init__(self,data,token=True):
    self.data = data
    if token:
      self.word2idx = {'<START>':1, '<END>':2, '<PAD>':0}
      self.idx2word = {1:'<START>', 2:'<END>', 0:'<PAD>'}
      self.idx = 2

    else:
      self.word2idx = {'<PAD>':0, '<END>':1}
      self.idx2word = {0:'<PAD>', 1:'<END>'}
      self.idx = 1

    self.x = []
    self.create()
    self.vocab_size = self.idx + 1

  def create(self):
    max_len = 0;
    for sentence in  self.data:
      max_len = max(max_len, len(sentence))
      for word in sentence:
        if self.word2idx.get(word) is None:
          self.idx += 1
          self.word2idx[word] = self.idx
          self.idx2word[self.idx] = word
    
    for sentence in self.data:
      sent = []
      for word in sentence:
        sent.append(self.word2idx[word])
      
      for i in range(len(sentence),max_len+1):
        sent.append(0)
      
      self.x.append(torch.Tensor(sent))

    

In [14]:
English_vocab = vocab(data[:,0],token=False)
Hindi_vocab = vocab(data[:,1],token=True)

In [15]:
for idx in Hindi_vocab.x[2]:
  print(Hindi_vocab.idx2word[int(idx)],end=' ')

<START> यह प्रतिशत भारत में हिन्दुओं प्रतिशत से अधिक है <END> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <P

In [16]:
class parallelData(Dataset):

  def __init__(self):
    self.x = English_vocab.x
    self.y = Hindi_vocab.x

  def __getitem__(self,i):
    return self.x[i], self.y[i]
  
  def __len__(self):
    return len(self.x)


In [17]:
dataset = parallelData()

In [18]:
a = dataset[0][0].shape[0]
b = dataset[0][1].shape[0]
for i in range(len(dataset)):
  if a != dataset[i][0].shape[0] or b != dataset[i][1].shape[0]:
    print(a,dataset[i][0].shape[0],b,dataset[i][1].shape[0])

In [19]:
#Model preparation

class encoder(nn.Module):

  def __init__(self, input_size, embedding_size, hidden_size, layers):
    '''
    input_size = size of vocab
    embedding_size = embedding dim
    hidden_size = hidden state size
    layer = num of layers of lstms
    '''
    super().__init__()
    self.embed = nn.Embedding(num_embeddings=input_size, embedding_dim=embedding_size) # output size = (*,embedding_size)
    self.lstm = nn.LSTM(input_size=embedding_size, hidden_size= hidden_size, num_layers=layers, batch_first = True)

  def forward(self,x):
    '''
    x shape = [batch_size, sentence]
    one complete sentence represents a "sequence"
    '''
    x = self.embed(x) # shape [batch_size,  sentence, embed_size]
    output, (hidden_state, cell_state) = self.lstm(x) #shape [batch_size, seq_len, hidden_size]

    return hidden_state, cell_state


class decoder(nn.Module):

  def __init__(self,input_size, embedding_size, hidden_size, layers):
    '''
    same configuration as encoder
    here input_size = size of hindi vocab
    '''
    super().__init__()
    self.embed = nn.Embedding(num_embeddings=input_size, embedding_dim=embedding_size) # output size = (*,embedding_size)
    self.lstm = nn.LSTM(input_size=embedding_size, hidden_size= hidden_size, num_layers=layers, batch_first = True)
    self.fc = nn.Linear(in_features=hidden_size, out_features=input_size) #since output would be prob distribution among hindi vocab therefore out_feature=input_size

  def forward(self,x,hidden_state, cell_state):
    '''
    to have control over output we have to take sentence as word by word
    therefore seq_len would be 1 as input is  one word not the whole sentence
    x = [batch_size] ->required-> [batch_size, 1] (1 is seq_len)
    '''
    # print(x.shape)
    x = x.reshape(-1,1) # shape [batch, 1]
    # print(x.shape)
    x = self.embed(x) # shape [batch, 1, embed_dim]

    output, (hidden_state, cell_state) = self.lstm(x, (hidden_state, cell_state)) # shape output=>[batch, 1, hidden_size], hidden=>[layers, batch, hidden_size]
    output = self.fc(output) # shape [batch, 1, hindi_vocab_size]
    
    #just for removing extra dim
    output = output.squeeze(dim=1) #shape [batch, hindi_vocab_size]

    return output, hidden_state, cell_state



class seq2seq(nn.Module):
  def __init__(self, encoder, decoder):
    super().__init__()
    self.encoder = encoder
    self.decoder = decoder


  def forward(self, input, target, teaching_force=0.5):
    '''
    input = batch of english sentences[batch, sentece(padded)]
    target = batch of hindi sentences [batch, sentence(padded)] 
    '''
    batch_size = input.shape[0]
    seq_len = target.shape[1]
    hindi_vocab_size = Hindi_vocab.vocab_size

    output = torch.zeros((seq_len, batch_size, hindi_vocab_size)).to(device)

    hidden, cell = self.encoder(input)
    target = target.permute(1,0) # shape [seq, batch]
    x = target[0] # <START> token

    for i in range(1, seq_len):
      out, hidden, cell = self.decoder(x, hidden, cell) #out shape = [batch, vocab_size]
      output[i] = out
      decoder_guess = out.argmax(1)# taking the word with max value(confidence)  shape = [batch of words]

      if random.random() < teaching_force:
        x = target[i]
      else:
        x =  decoder_guess
    
    return output  #shape[seq_len, batch_size, vocab_size]


In [20]:
##training
epochs = 15
learning_rate = 0.001
batch_size = 64

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
embedding_size = 300
hidden_size = 1024
layers = 1


In [21]:
loader = DataLoader(dataset, batch_size=batch_size, shuffle=True)

In [22]:
it = iter(loader)
x,y = next(it)
print(x.shape,y.shape)

torch.Size([64, 214]) torch.Size([64, 229])


In [23]:
ENC = encoder(English_vocab.vocab_size, embedding_size, hidden_size, layers).to(device)
DE = decoder(Hindi_vocab.vocab_size, embedding_size, hidden_size, layers).to(device)
model = seq2seq(ENC,DE).to(device)
optimizer = optim.Adam(model.parameters(),lr=learning_rate)
criterion = nn.CrossEntropyLoss(ignore_index=0)


In [24]:
for epoch in range(epochs):
  for id,(x,y) in (enumerate(tqdm(loader))):
    x = x.long().to(device)
    y = y.long().to(device)#[batch,seq]

    output = model(x,y)# [seq, batch, vocab]
    output = output[1:].reshape(-1,output.shape[2])
    y = y.permute(1,0)#[seq, batch]
    y = y[1:].reshape(-1)

    optimizer.zero_grad()
    loss = criterion(output,y)

    loss.backward()
    optimizer.step()

    if id%20 == 0:
      print(f'[{epoch+1}/{epochs}] loss=>{loss.item()}')
  
  

[1/15] loss=>10.009783744812012
[1/15] loss=>7.600383281707764
[1/15] loss=>7.5702714920043945
[1/15] loss=>7.233171463012695
[1/15] loss=>7.477302551269531
[1/15] loss=>7.559071063995361
[1/15] loss=>7.192291736602783
[1/15] loss=>7.277509689331055



[2/15] loss=>6.976804256439209
[2/15] loss=>6.964381694793701
[2/15] loss=>6.928804874420166
[2/15] loss=>7.073829650878906
[2/15] loss=>6.8861236572265625
[2/15] loss=>6.909426689147949
[2/15] loss=>7.011086463928223
[2/15] loss=>6.951757431030273



[3/15] loss=>6.903759002685547
[3/15] loss=>6.738678455352783
[3/15] loss=>6.921097755432129
[3/15] loss=>6.812922477722168
[3/15] loss=>6.69527006149292
[3/15] loss=>6.771994113922119
[3/15] loss=>6.998824119567871
[3/15] loss=>6.773809432983398



[4/15] loss=>6.708806037902832
[4/15] loss=>6.620732307434082
[4/15] loss=>6.730179309844971
[4/15] loss=>6.70827054977417
[4/15] loss=>6.197701930999756
[4/15] loss=>6.703220844268799
[4/15] loss=>6.892098903656006
[4/15] loss=>6.706884860992432



[5/15] loss=>6.2407708168029785
[5/15] loss=>6.387729644775391
[5/15] loss=>6.205647945404053
[5/15] loss=>6.770133018493652
[5/15] loss=>6.49815559387207
[5/15] loss=>6.23178768157959
[5/15] loss=>6.257687568664551
[5/15] loss=>6.443593502044678



...

[11/15] loss=>5.204305648803711
[11/15] loss=>4.494490623474121
[11/15] loss=>5.129806995391846
[11/15] loss=>5.285857200622559
[11/15] loss=>5.307097911834717
[11/15] loss=>4.84854793548584
[11/15] loss=>4.762656211853027
[11/15] loss=>5.168835639953613



[12/15] loss=>5.220032215118408
[12/15] loss=>5.356147766113281
[12/15] loss=>4.879451274871826
[12/15] loss=>5.409126281738281
[12/15] loss=>5.41495943069458
[12/15] loss=>5.021242618560791
[12/15] loss=>5.12440299987793
[12/15] loss=>4.543549060821533



[13/15] loss=>4.58946418762207
[13/15] loss=>4.28959846496582
[13/15] loss=>4.000698089599609
[13/15] loss=>4.64610481262207
[13/15] loss=>4.380892753601074
[13/15] loss=>4.357753276824951
[13/15] loss=>4.308812141418457
[13/15] loss=>4.724696636199951



[14/15] loss=>4.267877101898193
[14/15] loss=>3.9207816123962402
[14/15] loss=>4.51660680770874
[14/15] loss=>4.376156330108643
[14/15] loss=>4.309987545013428
[14/15] loss=>4.158724308013916
[14/15] loss=>4.411581516265869
[14/15] loss=>4.858973026275635



[15/15] loss=>4.2933807373046875
[15/15] loss=>4.388154029846191
[15/15] loss=>3.907578468322754
[15/15] loss=>4.905379772186279
[15/15] loss=>4.661643981933594
[15/15] loss=>4.277120590209961
[15/15] loss=>4.76821231842041
[15/15] loss=>4.033880233764648



###if you have access to powerful GPU you can run this for 40-50 epochs to achieve lesser loss. Here I have run this for 15 epochs only that took me around 2-2.5 hrs

In [93]:
def predict(x):
    for idx in x:
      if idx == 0:
        break
      print(English_vocab.idx2word[int(idx)],end=' ')
    
    print()

    x = x.long().reshape(1,-1).to(device)
    ans = translate(x)
    for id in ans:
      print(Hindi_vocab.idx2word[id],end=' ');

    print()
    

    


In [94]:
def translate(input):
      #input = batch of english sentences[batch, sentece(padded)]
      with torch.no_grad():
        guess = []
        hidden, cell = model.encoder(input)
        # x = torch.ones((1)).float().to(device) # <START> token
        x = torch.ones((1)).long().to(device)
        while True:
          out, hidden, cell = model.decoder(x, hidden, cell) #out shape = [batch, vocab_size]
          x = out.argmax(1)# taking the word with max value(confidence)  shape = [batch of words]
          guess.append(int(x[0].detach().cpu()))

          if x == 2:
            break

      return guess

In [60]:
predict(dataset[50][0])

category information technology <END> 
श्रेणी <END> 

In [63]:
predict(dataset[100][0])

class united states of america <END> 
श्रेणी के हिन्दी टाइपिंग <END> 

In [83]:
predict(dataset[71][0])

he often spoke to us of his own great longing to meet him <END> 
वे अपने को पचा के लिए कि वह सामन्य पर बैठे हुए <END> 

In [92]:
predict(dataset[random.randint(0,10000)][0])

and depressing ones such as the fact that <END> 
और यह में के लिए <END> 

In [103]:
for i in range(15):
  predict(dataset[random.randint(0,10000)][0])

they have over a billion people in this country <END> 
इस के में के के और के के <END> 
all others books are called smrtis <END> 
इस के के के <END> 
unionseek com d ioo exe <END> 
श्रेणी के <END> 
were located near ventilation ducts <END> 
इस के के के में <END> 
of course these mountains differ very much in cold and heat <END> 
वे इस कथानक में उन्होंने कहा के एम्स में से में <END> 
outside network <END> 
बाहरी कड़ियाँ <END> 
for those who march on the path to liberation or those who study philosophy and theology and who desire abstract truth which they call sara are entirely free from worshipping anything but god alone and would never dream of worshipping an image manufactured to represent him <END> 
इस के के के के और जो कि के के से दूर तक कि कि कि वह अपनी सल्तनत के साथ साथ ही के लिए और के लिए में के लिए की है और जो एक दूसरे के के के लिए में के लिए में के लिए <END> 
it s very difficult for a politician <END> 
यह एक है है कि <END> 
which we run every two weeks of the year in formula 1 <E

#as we can see its not performing well because dataset in itself is very diverse thats why its very difficult to learn parameters(patterns) in just 15 epochs
#one more intresting thing to observe is that, for larger lengths of sentences our model is struggling therefor we could also use attention model to get better results than vanilla seq2seq model